In [ ]:
import sys
import os

In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
print(sys.path)

In [ ]:
sys.path
import math
import gillespy2
from gillespy2 import BasicHybridSolver

In [ ]:

class SimpleHybridModel(gillespy2.Model):
     def __init__(self, parameter_values=None):
            #initialize Model
            gillespy2.Model.__init__(self, name="Simple_Hybrid_Model")

            
            #Species
            A = gillespy2.Species(name='stoch_species', initial_value=0)
            V = gillespy2.Species(name='det_species', initial_value=10, deterministic=True)

            self.add_species([A, V])
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 0.05)
            rate2 = gillespy2.Parameter(name='rate2', expression= 0.5)
            rate_rule1 = gillespy2.RateRule(V, "math.cos(t)")
            self.add_parameter([rate1, rate2])
            self.add_rate_rule(rate_rule1)
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={}, products={A:1},
                   rate="rate1 * V")
            
            r2 = gillespy2.Reaction(name="r2",reactants={A:1}, products={},
                    rate=rate2)
            
            self.add_reaction([r1,r2])

In [ ]:
model = SimpleHybridModel()